In [1]:
import utils.utils as utils
import utils.datasets as datasets

import os
import numpy as np
from numba import jit
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.nn.init import xavier_normal_
import torch.optim as optim
from sklearn.metrics import f1_score, confusion_matrix
from skimage.transform import resize
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import geotorch
import itertools
import kornia as K
from scipy import ndimage
from tqdm import tqdm
from typing import Callable, List, Tuple

pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

In [2]:
import random

random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
df, input_data = datasets.cross_dataset(resize=28, optical_flow=True)

In [5]:
aus = df.loc[:, "AU1":].sum(0)[df.loc[:, "AU1":].sum(0) > 10].index.tolist()
# Remove AU45
aus.remove("AU45")
aus.remove("AU18")
aus.remove("AU26")
aus.remove("AU38")

In [5]:
from models.models import OffApexNet
model = OffApexNet(task_num=len(aus))

In [6]:
input_data = input_data[:, :2]

In [8]:
import utils.train_eval as te
outputs_list = te.cross_dataset_validation(model, input_data, df)

[autoreload of utils.utils failed: Traceback (most recent call last):
  File "/home/tvaranka/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/tvaranka/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/tvaranka/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/tvaranka/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/tvaranka/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/tvaranka/.local/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  Fi

Dataset: casme, n=189 |train_f1: 0.66799 |test_f1: 0.59173
Test F1 per AU: [('AU1', 0.72688), ('AU2', 0.89138), ('AU4', 0.78034), ('AU5', 0.49734), ('AU6', 0.4933), ('AU7', 0.52561), ('AU9', 0.44083), ('AU10', 0.49465), ('AU12', 0.62828), ('AU14', 0.46761), ('AU15', 0.60843), ('AU17', 0.63979), ('AU18', 0.48501), ('AU20', 0.49734), ('AU24', 0.49465), ('AU25', 0.4933), ('AU26', 0.48641), ('AU38', 1.0)]

Dataset: casme2, n=256 |train_f1: 0.69113 |test_f1: 0.60919
Test F1 per AU: [('AU1', 0.95719), ('AU2', 0.85331), ('AU4', 0.9413), ('AU5', 0.49507), ('AU6', 0.54839), ('AU7', 0.66389), ('AU9', 0.55359), ('AU10', 0.54367), ('AU12', 0.65273), ('AU14', 0.47216), ('AU15', 0.59694), ('AU17', 0.7), ('AU18', 0.49902), ('AU20', 0.49902), ('AU24', 0.49804), ('AU25', 0.49804), ('AU26', 0.49507), ('AU38', 0.49804)]

Dataset: samm, n=159 |train_f1: 0.69096 |test_f1: 0.63152
Test F1 per AU: [('AU1', 0.70556), ('AU2', 0.82627), ('AU4', 0.84828), ('AU5', 0.48039), ('AU6', 0.48875), ('AU7', 0.83298), ('A

In [33]:
dataset_df = pd.concat(
    [
        df.groupby("dataset")[aus].sum(),
        pd.DataFrame(df.groupby("dataset")[aus].sum().sum(0)).T,
    ]
)
dataset_df["total"] = dataset_df.sum(1)
dataset_df = dataset_df.rename({0: "Total"})
dataset_df

,AU1,AU2,AU4,AU5,AU6,AU7,AU9,AU10,AU12,AU14,AU15,AU17,AU20,AU24,AU25,total
casme,23,17,70,0,1,4,40,3,9,23,14,13,2,4,5,228
casme2,26,22,130,2,13,39,13,16,34,27,16,25,1,2,2,368
fourDmicro,46,54,102,8,25,89,1,2,53,4,6,11,2,9,1,413
mmew,50,41,109,70,10,47,8,38,37,17,6,11,8,3,5,460
samm,6,18,23,10,3,46,5,6,28,13,4,6,7,9,7,191
Total,151,152,434,90,52,225,67,65,161,84,46,66,20,27,20,1660


AU1,AU2,AU4,AU6,AU7,AU10,AU12,AU15,AU23,AU24,AU25,AU26

In [86]:
def calculate_optical_flow(onset: np.ndarray, apex: np.ndarray) -> np.ndarray:
    # flow = cv2.calcOpticalFlowFarneback(onset, apex, flow=None,
    #                                pyr_scale=0.8, levels=15, winsize=5,
    #                                iterations=10,
    #                                poly_n=5, poly_sigma=0, flags=10)
    optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
    flow = optical_flow.calc(onset, apex, None)
    u = flow[..., 0]
    v = flow[..., 1]
    ux = ndimage.sobel(u, axis=0, mode="constant")
    uy = ndimage.sobel(u, axis=1, mode="constant")
    vx = ndimage.sobel(v, axis=0, mode="constant")
    vy = ndimage.sobel(v, axis=1, mode="constant")
    grad = np.sqrt(ux**2 + vy**2 + 0.5 * (uy**2 + vx**2))
    frames = np.concatenate([flow, np.expand_dims(grad, 2)], 2)
    frames = (frames - frames.min((0, 1))) / (frames.max((0, 1)) - frames.min((0, 1)))
    return frames

In [87]:
n_videos = len(data)
of_frames = np.empty((n_videos, 3, 64, 64))
for i, video in enumerate(tqdm(data)):
    flow = calculate_optical_flow(video[0], video[df.loc[i, "apexf"]])
    of_frames[i] = resize(flow, (64, 64, 3)).transpose(2, 0, 1)

  2%|▏         | 28/1171 [02:56<2:38:14,  8.31s/it]/home/tvaranka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
100%|██████████| 1171/1171 [2:43:07<00:00,  8.36s/it] 


In [ ]:
# Test that onset and offset frames in df correspond to the ones in files

secrets of OF
All AUs:  [('AU1', 0.8404276225287899), ('AU2', 0.7960803677741352), ('AU4', 0.8906447061614922), ('AU6', 0.5242435289828655), ('AU7', 0.7294963034616182), ('AU9', 0.618928652542098), ('AU10', 0.5426433298286846), ('AU12', 0.7222420254231772), ('AU14', 0.6625211585480535), ('AU15', 0.7993002422238455), ('AU17', 0.7373188405797102), ('AU18', 0.5476925899874614), ('AU20', 0.49535962877030165), ('AU24', 0.49300699300699297), ('AU25', 0.49447995351539803), ('AU26', 0.49477351916376305), ('AU5', 0.493891797556719)]
Mean f1:  0.6401794858855944

dualtvl1
All AUs:  [('AU1', 0.48669477310848175), ('AU2', 0.47437389542652697), ('AU4', 0.4770888678763864), ('AU5', 0.4938988959907031), ('AU6', 0.4870435806831566), ('AU7', 0.43770174306003873), ('AU9', 0.48247177658942364), ('AU10', 0.4909409701928696), ('AU12', 0.47095950704225353), ('AU14', 0.4784431137724551), ('AU15', 0.5336759824392333), ('AU17', 0.51844950446392), ('AU18', 0.4953650057937428), ('AU20', 0.49653179190751445), ('AU24', 0.49271986022131625), ('AU25', 0.4953650057937427), ('AU26', 0.495949074074074)]
Mean f1:  0.4886866675550494

all 5 datasets - sample 222 from casme2 as no OF
secrets of OF
Dataset: casme, n=189 | train_f1: 0.99912 | test_f1: 0.62678
Test F1 per AU: [('AU1', 0.84953), ('AU2', 0.88379), ('AU4', 0.79301), ('AU5', 0.48919), ('AU6', 0.48641), ('AU7', 0.49047), ('AU9', 0.49107), ('AU10', 0.63612), ('AU12', 0.67227), ('AU14', 0.72688), ('AU15', 0.78883), ('AU17', 0.66124), ('AU18', 0.64152), ('AU20', 0.496), ('AU24', 0.69598), ('AU25', 0.49194), ('AU26', 0.48919), ('AU38', 0.49867)]

Dataset: casme2, n=255 | train_f1: 1.0 | test_f1: 0.6476
Test F1 per AU: [('AU1', 0.92218), ('AU2', 0.86314), ('AU4', 0.95294), ('AU5', 0.49704), ('AU6', 0.64466), ('AU7', 0.72508), ('AU9', 0.60446), ('AU10', 0.53532), ('AU12', 0.74986), ('AU14', 0.68722), ('AU15', 0.68776), ('AU17', 0.80096), ('AU18', 0.49803), ('AU20', 0.49605), ('AU24', 0.49803), ('AU25', 0.49803), ('AU26', 0.49803), ('AU38', 0.49803)]

Dataset: samm, n=159 | train_f1: 0.99976 | test_f1: 0.67075
Test F1 per AU: [('AU1', 0.73675), ('AU2', 0.8747), ('AU4', 0.79965), ('AU5', 0.73675), ('AU6', 0.49201), ('AU7', 0.86854), ('AU9', 0.49201), ('AU10', 0.82848), ('AU12', 0.72412), ('AU14', 0.58018), ('AU15', 0.83013), ('AU17', 0.67042), ('AU18', 0.49363), ('AU20', 0.48875), ('AU24', 0.58701), ('AU25', 0.48875), ('AU26', 0.63482), ('AU38', 0.74682)]

Dataset: fourDmicro, n=267 | train_f1: 0.9988 | test_f1: 0.66555
Test F1 per AU: [('AU1', 0.87033), ('AU2', 0.81615), ('AU4', 0.91518), ('AU5', 0.48054), ('AU6', 0.5854), ('AU7', 0.68489), ('AU9', 0.49906), ('AU10', 0.49527), ('AU12', 0.65881), ('AU14', 0.62274), ('AU15', 0.63811), ('AU17', 0.72659), ('AU18', 1.0), ('AU20', 0.49718), ('AU24', 0.49143), ('AU25', 0.49906), ('AU26', 0.49906), ('AU38', 1.0)]

Dataset: mmew, n=300 | train_f1: 1.0 | test_f1: 0.6567
Test F1 per AU: [('AU1', 0.9062), ('AU2', 0.84462), ('AU4', 0.88116), ('AU5', 0.43396), ('AU6', 0.49153), ('AU7', 0.75417), ('AU9', 0.53448), ('AU10', 0.49107), ('AU12', 0.68519), ('AU14', 0.68142), ('AU15', 0.54181), ('AU17', 0.71689), ('AU18', 0.59322), ('AU20', 0.49066), ('AU24', 0.74832), ('AU25', 0.6633), ('AU26', 0.74792), ('AU38', 0.61473)]

All AUs:  [('AU1', 0.88261), ('AU2', 0.84916), ('AU4', 0.88785), ('AU5', 0.50967), ('AU6', 0.57011), ('AU7', 0.73319), ('AU9', 0.54219), ('AU10', 0.55996), ('AU12', 0.70309), ('AU14', 0.67494), ('AU15', 0.68771), ('AU17', 0.73576), ('AU18', 0.59847), ('AU20', 0.49394), ('AU24', 0.59481), ('AU25', 0.54114), ('AU26', 0.67146), ('AU38', 0.62677)]
Mean f1:  0.6590460779085798


resnet18 pretrained
All AUs:  [('AU1', 0.8571269075912887), ('AU2', 0.8332019683445882), ('AU4', 0.8957803115137942), ('AU5', 0.5192725130240635), ('AU6', 0.539229746097088), ('AU7', 0.69580101399662), ('AU9', 0.5184270516717325), ('AU10', 0.5376139111028455), ('AU12', 0.6584340514976353), ('AU14', 0.5923957886398891), ('AU15', 0.6407894736842106), ('AU17', 0.7124469770066814), ('AU18', 0.6023850716757291), ('AU20', 0.49547218628719275), ('AU24', 0.5286411316193915), ('AU25', 0.49547218628719275), ('AU26', 0.561038961038961), ('AU38', 0.49612403100775193)]
Mean f1:  0.6210918490048142

resnet18
All AUs:  [('AU1', 0.8398003033367037), ('AU2', 0.8174410540915396), ('AU4', 0.8905428932955739), ('AU5', 0.5211225069640096), ('AU6', 0.5218421893460347), ('AU7', 0.7166788677232829), ('AU9', 0.4832155477031802), ('AU10', 0.5576702888375208), ('AU12', 0.6495192307692308), ('AU14', 0.5695945945945947), ('AU15', 0.6158056042031524), ('AU17', 0.7042344797790147), ('AU18', 0.533275713050994), ('AU20', 0.49547218628719275), ('AU24', 0.4941634241245136), ('AU25', 0.49547218628719275), ('AU26', 0.5881174176776429), ('AU38', 0.4959069366652305)]
Mean f1:  0.610548634707589

resnet8
All AUs:  [('AU1', 0.8548477141616524), ('AU2', 0.8093733466849489), ('AU4', 0.8903207259455541), ('AU5', 0.49244748412310696), ('AU6', 0.5202814003841202), ('AU7', 0.7043051218700412), ('AU9', 0.5076484186709951), ('AU10', 0.5121789100313613), ('AU12', 0.6716649887846836), ('AU14', 0.5794535734149581), ('AU15', 0.5975106329747311), ('AU17', 0.6627072349615701), ('AU18', 0.5350323974082074), ('AU20', 0.4952545297670406), ('AU24', 0.4939446366782007), ('AU25', 0.49503668536901163), ('AU26', 0.5850573353824329), ('AU38', 0.49612403100775193)]
Mean f1:  0.6057327315344649

In [2]:
from functools import partial

import utils.train_eval as te
import utils.models.models as models
import utils.datasets as datasets
from utils.latex_tools import results_to_latex


import torch
import torch.optim as optim

In [3]:
class BasicConfig:
    device = torch.device("cuda:0")
    epochs = 200
    # Optimizer
    learning_rate = 1e-2
    weight_decay = 1e-3
    optimizer = partial(optim.SGD, momentum=0.9)
    scheduler = None
    # Dataloader
    batch_size = 64
    train_transform = None
    test_transform = None
    action_units = ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10',
                    'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU24', 'AU25']

In [3]:
class SSSNetConfig(BasicConfig):
    model = models.SSSNet(h_dims=[32, 64, 256], task_num=len(BasicConfig.action_units))
    
df, input_data = datasets.cross_dataset(resize=64, optical_flow=True)
outputs_list = te.cross_dataset_validation(input_data, df, SSSNetConfig)
results_to_latex(outputs_list, df, SSSNetConfig.action_units)

Dataset: casme, n=189 | train_f1: 1.0 | test_f1: 0.64124
Test F1 per AU: [('AU1', 0.86021), ('AU2', 0.83841), ('AU4', 0.80475), ('AU5', 0.49465), ('AU6', 0.4933), ('AU7', 0.50142), ('AU9', 0.59432), ('AU10', 0.65574), ('AU12', 0.66216), ('AU14', 0.64449), ('AU15', 0.72269), ('AU17', 0.66124), ('AU20', 0.49734), ('AU24', 0.69598), ('AU25', 0.49194)]

Dataset: casme2, n=256 | train_f1: 1.0 | test_f1: 0.64041
Test F1 per AU: [('AU1', 0.94907), ('AU2', 0.86207), ('AU4', 0.94138), ('AU5', 0.49206), ('AU6', 0.48697), ('AU7', 0.71961), ('AU9', 0.55359), ('AU10', 0.54367), ('AU12', 0.6792), ('AU14', 0.67134), ('AU15', 0.59694), ('AU17', 0.61619), ('AU20', 0.49804), ('AU24', 0.49804), ('AU25', 0.49804)]

Dataset: samm, n=159 | train_f1: 1.0 | test_f1: 0.64851
Test F1 per AU: [('AU1', 0.64849), ('AU2', 0.83534), ('AU4', 0.84386), ('AU5', 0.56699), ('AU6', 0.48544), ('AU7', 0.80293), ('AU9', 0.74355), ('AU10', 0.74355), ('AU12', 0.64403), ('AU14', 0.61518), ('AU15', 0.49363), ('AU17', 0.7402), ('

In [13]:
class ModelConfig(BasicConfig):
    model = models.Resnet18(task_num=len(BasicConfig.action_units), pretrained=True)
    
df, input_data = datasets.cross_dataset(resize=112, optical_flow=True)
outputs_list = te.cross_dataset_validation(input_data, df, ModelConfig)
results_to_latex(outputs_list, df, ModelConfig.action_units)

Dataset: casme, n=189 | train_f1: 1.0 | test_f1: 0.58955
Test F1 per AU: [('AU1', 0.7753), ('AU2', 0.83193), ('AU4', 0.81375), ('AU5', 0.49057), ('AU6', 0.48501), ('AU7', 0.4958), ('AU9', 0.44083), ('AU10', 0.57865), ('AU12', 0.62108), ('AU14', 0.56552), ('AU15', 0.66124), ('AU17', 0.59964), ('AU20', 0.496), ('AU24', 0.49465), ('AU25', 0.4933)]

Dataset: casme2, n=256 | train_f1: 0.97797 | test_f1: 0.59821
Test F1 per AU: [('AU1', 0.87824), ('AU2', 0.825), ('AU4', 0.89382), ('AU5', 0.47216), ('AU6', 0.48178), ('AU7', 0.66816), ('AU9', 0.52827), ('AU10', 0.53936), ('AU12', 0.67135), ('AU14', 0.46998), ('AU15', 0.54367), ('AU17', 0.51126), ('AU20', 0.49902), ('AU24', 0.49804), ('AU25', 0.49307)]

Dataset: samm, n=159 | train_f1: 1.0 | test_f1: 0.63384
Test F1 per AU: [('AU1', 0.77585), ('AU2', 0.81404), ('AU4', 0.85763), ('AU5', 0.66859), ('AU6', 0.49363), ('AU7', 0.77334), ('AU9', 0.66026), ('AU10', 0.74355), ('AU12', 0.6203), ('AU14', 0.47351), ('AU15', 0.49363), ('AU17', 0.67042), ('A

In [22]:
class ModelConfig(BasicConfig):
    model = models.OffApexNet(task_num=len(BasicConfig.action_units))
    epochs = 1000
    learning_rate = 1e-4
    optimizer = optim.Adam
    
df, input_data = datasets.cross_dataset(resize=28, optical_flow=True)
input_data = input_data[:, :2]
outputs_list = te.cross_dataset_validation(input_data, df, ModelConfig)
results_to_latex(outputs_list, df, ModelConfig.action_units)

Dataset: casme, n=189 | train_f1: 0.99947 | test_f1: 0.62172
Test F1 per AU: [('AU1', 0.81416), ('AU2', 0.86518), ('AU4', 0.78034), ('AU5', 0.496), ('AU6', 0.4878), ('AU7', 0.54567), ('AU9', 0.55049), ('AU10', 0.7033), ('AU12', 0.62828), ('AU14', 0.60785), ('AU15', 0.66124), ('AU17', 0.70833), ('AU20', 0.49194), ('AU24', 0.4933), ('AU25', 0.49194)]

Dataset: casme2, n=256 | train_f1: 0.99513 | test_f1: 0.63055
Test F1 per AU: [('AU1', 0.9198), ('AU2', 0.87115), ('AU4', 0.87488), ('AU5', 0.48283), ('AU6', 0.54839), ('AU7', 0.67906), ('AU9', 0.58946), ('AU10', 0.53936), ('AU12', 0.6189), ('AU14', 0.5817), ('AU15', 0.59694), ('AU17', 0.66667), ('AU20', 0.49902), ('AU24', 0.49307), ('AU25', 0.49705)]

Dataset: samm, n=159 | train_f1: 0.99083 | test_f1: 0.63596
Test F1 per AU: [('AU1', 0.67987), ('AU2', 0.84785), ('AU4', 0.68267), ('AU5', 0.52889), ('AU6', 0.48377), ('AU7', 0.75579), ('AU9', 0.78089), ('AU10', 0.77585), ('AU12', 0.61877), ('AU14', 0.56662), ('AU15', 0.61532), ('AU17', 0.740

In [24]:
# Casme2 8 AUS cross-dataset significantly worse
np.mean([0.9198, 0.87115, 0.87488, 0.67906,  0.6189, 0.5817, 0.59694, 0.66667])

0.7261375

In [4]:
class ModelConfig(BasicConfig):
    model = models.STSTNet(task_num=len(BasicConfig.action_units))
    
df, input_data = datasets.cross_dataset(resize=28, optical_flow=True)
#input_data = input_data[:, :2]
outputs_list = te.cross_dataset_validation(input_data, df, ModelConfig)
results_to_latex(outputs_list, df, ModelConfig.action_units)

Dataset: casme, n=189 | train_f1: 0.8287 | test_f1: 0.60003
Test F1 per AU: [('AU1', 0.74145), ('AU2', 0.88429), ('AU4', 0.79699), ('AU5', 0.48919), ('AU6', 0.48919), ('AU7', 0.4879), ('AU9', 0.55856), ('AU10', 0.4878), ('AU12', 0.59032), ('AU14', 0.61429), ('AU15', 0.70833), ('AU17', 0.67369), ('AU20', 0.496), ('AU24', 0.4933), ('AU25', 0.48919)]

Dataset: casme2, n=256 | train_f1: 0.84474 | test_f1: 0.63358
Test F1 per AU: [('AU1', 0.92378), ('AU2', 0.84164), ('AU4', 0.92186), ('AU5', 0.49804), ('AU6', 0.60452), ('AU7', 0.69948), ('AU9', 0.48697), ('AU10', 0.53936), ('AU12', 0.64207), ('AU14', 0.47216), ('AU15', 0.76252), ('AU17', 0.61619), ('AU20', 0.49902), ('AU24', 0.49804), ('AU25', 0.49804)]

Dataset: samm, n=159 | train_f1: 0.84088 | test_f1: 0.60959
Test F1 per AU: [('AU1', 0.63946), ('AU2', 0.89295), ('AU4', 0.76526), ('AU5', 0.48377), ('AU6', 0.49201), ('AU7', 0.82316), ('AU9', 0.61532), ('AU10', 0.65359), ('AU12', 0.63216), ('AU14', 0.5242), ('AU15', 0.49363), ('AU17', 0.67

## Constant method

In [19]:
constant_outputs = [torch.zeros(i, len(BasicConfig.action_units)) for i in [189, 256, 159, 267, 300]]

In [20]:
results_to_latex(constant_outputs, df, BasicConfig.action_units)

AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU24', 'AU25', 'Average']
0.4655 & 0.4653 & 0.3863 & 0.48 & 0.4886 & 0.4469 & 0.4853 & 0.4857 & 0.4631 & 0.4814 & 0.49 & 0.4855 & 0.4957 & 0.4942 & 0.4957 & 0.4739

Datasets:  ['casme', 'casme2', 'samm', 'fourDmicro', 'mmew', 'Average']
0.5109 & 0.4723 & 0.4784 & 0.4705 & 0.4716 & 0.4807


# Multi label

In [7]:
from sklearn.metrics import f1_score
y_true = [[0, 0, 0], [1, 1, 1], [0, 1, 1]]
y_pred = [[0, 0, 0], [1, 1, 1], [1, 1, 0]]
f1_score(y_true, y_pred, average=None)

array([0.66666667, 1.        , 0.66666667])

In [9]:
import numpy as np
y_true = np.array(df.loc[df["dataset"] == "casme", SSSNetConfig.action_units])

In [10]:
y_pred = outputs_list[0].numpy()

In [14]:
f1_score(y_true, y_pred, average=None)

array([0.75      , 0.70588235, 0.75      , 0.        , 0.        ,
       0.07407407, 0.29166667, 0.33333333, 0.36363636, 0.35294118,
       0.47619048, 0.35294118, 0.        , 0.4       , 0.        ])

# Code to class

In [7]:
class SSSNetConfig(BasicConfig):
    epochs = 1
    model = models.SSSNet(h_dims=[32, 64, 256], task_num=len(BasicConfig.action_units))
    
df, input_data = datasets.cross_dataset(resize=64, optical_flow=True)
outputs_list = cross_dataset_validation(input_data, df, SSSNetConfig)
results_to_latex(outputs_list, df, SSSNetConfig.action_units)

Dataset: casme, n=189 | train_f1: 0.54619 | test_f1: 0.54221
Test F1 per AU: [('AU1', 0.58698), ('AU2', 0.53333), ('AU4', 0.63185), ('AU5', 1.0), ('AU6', 0.49867), ('AU7', 0.48641), ('AU9', 0.44083), ('AU10', 0.496), ('AU12', 0.54316), ('AU14', 0.46761), ('AU15', 0.48077), ('AU17', 0.48219), ('AU20', 0.49734), ('AU24', 0.49465), ('AU25', 0.4933)]

Dataset: casme2, n=256 | train_f1: 0.49242 | test_f1: 0.48436
Test F1 per AU: [('AU1', 0.47325), ('AU2', 0.47755), ('AU4', 0.35501), ('AU5', 0.49804), ('AU6', 0.48697), ('AU7', 0.60666), ('AU9', 0.48697), ('AU10', 0.48387), ('AU12', 0.47162), ('AU14', 0.47216), ('AU15', 0.48387), ('AU17', 0.47433), ('AU20', 0.49902), ('AU24', 0.49804), ('AU25', 0.49804)]

Dataset: samm, n=159 | train_f1: 0.54934 | test_f1: 0.53252
Test F1 per AU: [('AU1', 0.59605), ('AU2', 0.67339), ('AU4', 0.88134), ('AU5', 0.48377), ('AU6', 0.49524), ('AU7', 0.41544), ('AU9', 0.49201), ('AU10', 0.49038), ('AU12', 0.53448), ('AU14', 0.47869), ('AU15', 0.49363), ('AU17', 0.49

In [ ]:
df, input_data = datasets.cross_dataset(resize=64, optical_flow=True)

In [51]:
outputs_list = CrossDatasetValidation(input_data, df).validation()

Dataset: casme, n=189 | train_f1: 1.0 | test_f1: 0.64124
Test F1 per AU: [('AU1', 0.86021), ('AU2', 0.83841), ('AU4', 0.80475), ('AU5', 0.49465), ('AU6', 0.4933), ('AU7', 0.50142), ('AU9', 0.59432), ('AU10', 0.65574), ('AU12', 0.66216), ('AU14', 0.64449), ('AU15', 0.72269), ('AU17', 0.66124), ('AU20', 0.49734), ('AU24', 0.69598), ('AU25', 0.49194)]

Dataset: casme2, n=256 | train_f1: 1.0 | test_f1: 0.64041
Test F1 per AU: [('AU1', 0.94907), ('AU2', 0.86207), ('AU4', 0.94138), ('AU5', 0.49206), ('AU6', 0.48697), ('AU7', 0.71961), ('AU9', 0.55359), ('AU10', 0.54367), ('AU12', 0.6792), ('AU14', 0.67134), ('AU15', 0.59694), ('AU17', 0.61619), ('AU20', 0.49804), ('AU24', 0.49804), ('AU25', 0.49804)]

Dataset: samm, n=159 | train_f1: 1.0 | test_f1: 0.64851
Test F1 per AU: [('AU1', 0.64849), ('AU2', 0.83534), ('AU4', 0.84386), ('AU5', 0.56699), ('AU6', 0.48544), ('AU7', 0.80293), ('AU9', 0.74355), ('AU10', 0.74355), ('AU12', 0.64403), ('AU14', 0.61518), ('AU15', 0.49363), ('AU17', 0.7402), ('

In [ ]:
# Add class seperately test

In [50]:
class CrossDatasetValidation(Validation):
    def __init__(self, input_data: np.ndarray, df: pd.DataFrame):
        super().__init__()
        self.model = models.SSSNet(h_dims=[32, 64, 256], task_num=len(BasicConfig.action_units))
        self.epochs = 200
        self.df = df
        self.input_data = input_data
        number_of_tasks = len(self.action_units)
        self.criterion = utils.MultiTaskLoss(number_of_tasks)
        self.evaluation = utils.MultiTaskF1(number_of_tasks)
    
    def validation(self) -> List[np.ndarray]:
        """
        Cross dataset evaluation
        """
        utils.set_random_seeds()
        dataset_names = df["dataset"].unique()
        labels = np.concatenate([np.expand_dims(df[au], 1) for au in self.action_units], axis=1)
        number_of_tasks = labels.shape[1]
        outputs_list = []
        
        model = self.model.to(self.device)
        for dataset_name in dataset_names:
            train_data, train_labels, test_data, test_labels = split_data(
                self.df["dataset"], self.input_data, labels, dataset_name
            )
            train_loader = self.get_data_loader(train_data, train_labels, transform=self.train_transform, train=True)
            test_loader = self.get_data_loader(test_data, test_labels, transform=self.test_transform, train=False)
            self.model.apply(utils.reset_weights)
            self.optimizer = self.optimizer_obj(self.model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)

            self.train_model(train_loader)

            train_f1 = self.evaluate_model(train_loader)
            test_f1, outputs_test = self.evaluate_model(test_loader, test=True)
            outputs_list.append(outputs_test)
            print(
                f"Dataset: {dataset_name}, n={test_data.shape[0]} | "
                f"train_f1: {np.mean(train_f1):.5} | "
                f"test_f1: {np.mean(test_f1):.5}"
            )
            print(f"Test F1 per AU: {list(zip(self.action_units, np.around(test_f1, 5)))}\n")
        # Calculate total f1-scores
        predictions = torch.cat(outputs_list)
        f1_aus = self.evaluation(labels, predictions)
        print("All AUs: ", list(zip(self.action_units, np.around(f1_aus, 5))))
        print("Mean f1: ", np.mean(f1_aus))
        return outputs_list
    
    def evaluate_model(
        self,
        dataloader: torch.utils.data.DataLoader,
        test: bool = False,
    ) -> Union[List[float], Tuple[List[float], torch.tensor]]:
        """
        Evaluates the model given a dataloader and an evaluation function. Returns
        the evaluation result and if boolean test is set to true also the
        predictions.
        """
        self.model.eval()
        outputs_list = []
        labels_list = []
        for batch in dataloader:
            data_batch = batch[0].to(self.device)
            labels_batch = batch[1]
            outputs = self.model(data_batch.float())
            outputs_list.append([output.detach().cpu() for output in outputs])
            labels_list.append(labels_batch)
        self.model.train()
        predictions = self.outputs_list_to_predictions(outputs_list)
        labels = torch.cat(labels_list)
        result = self.evaluation(labels, predictions)
        if test:
            return result, predictions
        return result

In [23]:
from functools import partial
import numpy as np
import torch.nn as nn
import torch
import pandas as pd
import utils.utils as utils
from typing import Callable, Tuple, List, Union
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from abc import ABC, abstractmethod


class Validation(ABC):
    def __init__(self):
        self.device = torch.device("cuda:0")
        self.epochs = 200
        # Optimizer
        self.learning_rate = 1e-2
        self.weight_decay = 1e-3
        self.optimizer_obj = partial(optim.SGD, momentum=0.9)
        self.scheduler = None
        # Dataloader
        self.batch_size = 64
        self.train_transform = None
        self.test_transform = None
        self.action_units = ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10',
                        'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU24', 'AU25']
        self.criterion = nn.CrossEntropyLoss()
        
    @abstractmethod
    def validation(self):
        pass
    
    
    def split_data(
        self,
        split_column: pd.Series,
        data: np.ndarray,
        labels: np.ndarray,
        split_name: str
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Splits data based on the split_column and split_name. E.g., df["dataset"] == "smic"
        """
        train_idx = split_column[split_column != split_name].index
        test_idx = split_column[split_column == split_name].index
        return data[train_idx], labels[train_idx], data[test_idx], labels[test_idx]
    
    def get_data_loader(
        self,
        data: np.ndarray,
        labels: np.ndarray,
        transform,
        train: bool,
    ) -> torch.utils.data.DataLoader:
        """
        Constructs a pytorch dataloader form a given input data, labels and transform.
        """
        dataset = utils.MEData(data, labels, transform)
        dataloader = torch.utils.data.DataLoader(
            dataset, batch_size=self.batch_size, shuffle=train, num_workers=0, pin_memory=True
        )
        return dataloader
    
    def train_model(
        self,
        dataloader: torch.utils.data.DataLoader,
    ) -> None:
        for epoch in range(self.epochs):
            for batch in dataloader:
                data_batch, labels_batch = batch[0].to(self.device), batch[1].to(self.device)
                self.optimizer.zero_grad()

                outputs = self.model(data_batch.float())
                loss = self.criterion(outputs, labels_batch.long())
                loss.backward()
                self.optimizer.step()
    
    @abstractmethod
    def evaluate_model(
        self,
        dataloader: torch.utils.data.DataLoader,
        test: bool = False,
    ) -> Union[float, Tuple[float, torch.tensor]]:
        pass
    
    def outputs_list_to_predictions(
        self,
        outputs_list: List[List[torch.tensor]]
    ) -> torch.tensor:
        predictions = torch.cat(
            [
                torch.tensor(
                    [torch.max(au_output, 1)[1].tolist() for au_output in split_output]
                ).T
                for split_output in outputs_list
            ]
        )
        return predictions